#  Capstone Project Assigment


In [1]:
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup

For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1.Start by creating a new Notebook for this assignment.

2.Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [105]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
result = requests.get(url)

3.Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

4.More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

5.If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

6.Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making

In [196]:
soup = BeautifulSoup(result.content, 'html.parser')
table = soup.find('table')
trs = table.find_all('tr')
rows = []
for tr in trs:
    tds = tr.find_all('td')
    for i in tds:
        rows.append(i)
        
lst=[]
bor=""
postal=""
for row in rows:
    neigh=""
    if row.span.text[0:13] != 'Not assigned':
        try:
            vals=row.span.find_all('a')
            for v in range(len(vals)):
                if v == 0:
                    bor=row.span.find_all('a')[v].text
                else:
                    if neigh == "":
                        neigh=row.span.find_all('a')[v].text
                    else:
                        neigh=neigh+","+row.span.find_all('a')[v].text+","+neigh
            postal=row.b.text
            if neigh=="":
                lst.append([postal, bor, bor])
            else:
                lst.append([postal, bor, neigh])                
        except Exception as e:
            e

7. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [201]:
cols = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(lst, columns=cols)

8. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [202]:
df.shape

(103, 3)

Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [209]:
dfgeo = pd.read_csv("Geospatial_Coordinates.csv")
dfgeo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:


Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code

In [210]:
dfgeo.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df2 = pd.merge(df, dfgeo, on="PostalCode", how='left')

In [211]:
df2

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Old Mill,The Kingsway",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,Business reply mail,Business reply mail,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill,Sunnylea,Old Mill,Humber Bay,Old Mill...",43.636258,-79.498509
